In [2]:
import numpy as np 
import pandas as pd 
import shutil
import os
import zipfile
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from torch.autograd import Variable
from torchvision import transforms
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import copy
import tqdm
import time
import random
from PIL import Image

import albumentations
from albumentations import pytorch as AT


%matplotlib inline

In [3]:
hyper_batch_size = 128
hyper_num_workers = 10
hyper_img_size = 256
hyper_seed = 42
hyper_training_patience = 5
hyper_lr = 0.0005
hyper_sheduler_factor = 0.2
hyper_sheduler_patience = 2
hyper_train_factor = 0.7
hyper_val_test_factor = 0.5
hyper_n_epochs = 50

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(hyper_seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/CourseWorkData/train.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

In [ ]:
train_dir = '/content/train'
train_files = os.listdir(train_dir)
print(len(train_files))

In [ ]:
train_files[:10]

In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CourseWorkData/train_labels.csv', sep=',')
dataframe[dataframe.columns.values[0]] = dataframe[dataframe.columns.values[0]].str.strip('./train/')
print(dataframe)

In [ ]:
n_labels = dataframe[dataframe.columns.values[1]].nunique()
print(n_labels)

In [ ]:
labels_raw = dataframe[dataframe.columns.values[1]].unique()
labels_dict = dict(zip(labels_raw, list(range(n_labels))))
print(labels_dict)

In [ ]:
def labels_to_index(labels):
    indices = []
    for l in labels:
        indices.append(labels_dict[l])
    return indices

In [ ]:
indexed_labels = labels_to_index(dataframe[dataframe.columns.values[1]].values)
print(len(indexed_labels))

In [ ]:
sklearn_train_values, sklearn_test_val_values, sklearn_train_labels, sklearn_test_val_labels = train_test_split(
    dataframe.values,
    indexed_labels,
    test_size = 1 - hyper_train_factor,
    random_state = hyper_seed,
    stratify = indexed_labels
)

sklearn_test_values, sklearn_val_values, sklearn_test_labels, sklearn_val_labels = train_test_split(
    sklearn_test_val_values,
    sklearn_test_val_labels,
    test_size = hyper_val_test_factor,
    random_state = hyper_seed,
    stratify = sklearn_test_val_labels
)

print(len(sklearn_train_values), len(sklearn_test_values), len(sklearn_val_values))
print(len(sklearn_train_values) + len(sklearn_test_values) + len(sklearn_val_values))

In [ ]:
train_dataframe = pd.DataFrame(sklearn_train_values, columns = dataframe.columns.values)
val_dataframe = pd.DataFrame(sklearn_val_values, columns = dataframe.columns.values)
test_dataframe = pd.DataFrame(sklearn_test_values, columns = dataframe.columns.values)

In [ ]:
dataframe_series = dataframe.set_index(dataframe.columns.values[0]).squeeze()
print(dataframe_series)

In [ ]:
train_labels = labels_to_index(train_dataframe[train_dataframe.columns.values[1]].values)
class_sample_count = np.unique(train_labels, return_counts=True)[1]
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in train_labels])
samples_weight = torch.from_numpy(samples_weight)
print(samples_weight)

In [ ]:
sampler = WeightedRandomSampler(
    weights = samples_weight,
    num_samples = len(samples_weight),
    replacement = True
)

In [ ]:
class SportsDataset(Dataset):
    def __init__(self, file_list, dir, transform=None):
        self.file_list = file_list
        self.dir = dir
        self.transform = transform

            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        image = cv2.imread(os.path.join(self.dir, self.file_list[idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.label = labels_dict[dataframe_series[self.file_list[idx]]]
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, self.label

In [ ]:
data_transforms = albumentations.Compose([
    albumentations.Resize(hyper_img_size, hyper_img_size),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RandomBrightness(),
    albumentations.ChannelShuffle(p=0.5),
    albumentations.ShiftScaleRotate(rotate_limit=45, scale_limit=0.10),
    albumentations.HueSaturationValue(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])

data_transforms_test = albumentations.Compose([
    albumentations.Resize(hyper_img_size, hyper_img_size),
    albumentations.HorizontalFlip(),
    albumentations.RandomRotate90(),
    albumentations.Normalize(),
    AT.ToTensor()
    ])

In [ ]:
train_set = SportsDataset(
    file_list = train_dataframe[train_dataframe.columns.values[0]].values, 
    dir = train_dir, 
    transform = data_transforms
)

val_set = SportsDataset(
    file_list = val_dataframe[val_dataframe.columns.values[0]].values,
    dir = train_dir,
    transform = data_transforms
)

test_set = SportsDataset(
    file_list = test_dataframe[test_dataframe.columns.values[0]].values,
    dir = train_dir,
    transform = data_transforms_test
)

In [ ]:
train_loader = DataLoader(
    dataset = train_set,
    batch_size = hyper_batch_size,
    sampler = sampler,
    shuffle = False,
    pin_memory = True    
)

valid_loader = DataLoader(
    dataset = val_set,
    batch_size = hyper_batch_size,
    shuffle = True,
    pin_memory = True    
)

test_loader = DataLoader(
    dataset = test_set,
    batch_size = hyper_batch_size,
    shuffle = True
)

In [ ]:
samples, targets = next(iter(train_loader))
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
print(targets[:24])
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))

In [ ]:
model = torchvision.models.resnet101(pretrained=True, progress=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(model.fc.in_features, n_labels)

In [ ]:
def train_model(training_model, train_loader, valid_loader, criterion, optimizer, n_epochs):
    training_model.to(device)
    valid_loss_min = np.Inf
    patience = 5
    p = 0
    stop = False
    total_train_loss = []
    total_val_loss = []

    for epoch in range(1, n_epochs+1):
        print(time.ctime(), 'Epoch:', epoch)
        training_model.train()
        train_loss = []
        for batch_i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device).long()
            optimizer.zero_grad()
            output = training_model(data)
            loss = criterion(output, target)
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
          
        training_model.eval()
        val_loss = []
        for batch_i, (data, target) in enumerate(valid_loader):
            data, target = data.to(device), target.to(device).long()
            output = training_model(data)
            loss = criterion(output, target)
            val_loss.append(loss.item())

        print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}, valid loss: {np.mean(val_loss):.4f}.')

        total_train_loss.append(np.mean(train_loss))
        total_val_loss.append(np.mean(val_loss))

        valid_loss = np.mean(val_loss)
        scheduler.step(valid_loss)
        if valid_loss <= valid_loss_min:
            print(
                'Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                    valid_loss_min,
                    valid_loss
                )
            )
            torch.save(training_model.state_dict(), 'model.pt')
            valid_loss_min = valid_loss
            p = 0
        
        if valid_loss > valid_loss_min:
            p += 1
            print(f'{p} epochs of increasing val loss')
            if p > patience:
                print('Stopping training')
                stop = True
                break        

        if stop:
            break
    return training_model, total_train_loss, total_val_loss

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_lr, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=hyper_sheduler_factor, patience=hyper_sheduler_patience)

In [ ]:
trained_model, train_loss, val_loss = train_model(
    training_model = model, 
    train_loader = train_loader, 
    valid_loader = valid_loader, 
    criterion = criterion, 
    optimizer = optimizer, 
    n_epochs = hyper_n_epochs
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.plot(train_loss)
ax2.plot(val_loss)
ax1.set_ylabel("Train Loss")
ax1.set_xlabel("Epochs")
ax2.set_ylabel("Validaation Loss")
ax2.set_xlabel("Epochs")

In [ ]:
model.state_dict(torch.load('/content/model.pt'))
print('Success')

In [ ]:
total_preds = 0
correct_preds = 0
model.to(device)
model.eval()
pred_list = []
for sample, targets in test_loader:
    with torch.no_grad():
        sample = sample.to(device)
        output = model(sample)
        pred = torch.sigmoid(output)
        pred = pred.cpu().argmax(dim=1)
        pred_list = torch.empty(0)
        pred_list = torch.cat((pred_list, pred))
        total_preds += len(pred_list)
        correct_preds += (targets == pred_list).sum()

accuracy = 100.0 * correct_preds.float().item() / float(total_preds)
print(f'Accuracy: {accuracy}%')

In [ ]:
sample, _ = next(iter(test_loader))
sample = sample.to(device)
model.to(device)
output = model(sample)
pred = torch.sigmoid(output)
pred = pred.cpu().detach().numpy()
print([p.argmax() for p in pred])
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(sample[:24])
np_grid_imgs = grid_imgs.cpu().numpy()
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))